In [15]:
import os
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

from sklearn.model_selection import train_test_split
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Convolution2D, Dropout
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.metrics import f1_score
from PIL import Image

from constant_values import *
from img_preprocessing import *
from LIDAR_model import *
from im_postprocess import *

ModuleNotFoundError: No module named 'im_processing'

In [2]:
tr_imgs = data_augmentation("../Data/training/images/")      #100 images --> 2400 images
gt_imgs = data_augmentation("../Data/training/groundtruth/") #100 images --> 2400 images

Image 100/100 is being processed ... Shuffle data ...
Image 100/100 is being processed ... Shuffle data ...


In [3]:
# Extract each images into patches (numpy arrays).
img = extract_data(tr_imgs, TRAINING_SIZE)  #shape: ((400/16)^2 * TRAINING_SIZE, 16, 16, 3)
gt = extract_labels(gt_imgs, TRAINING_SIZE) #shape: ((400/16)^2 * TRAINING_SIZE, 2)

In [4]:
#Test model simple

# Create model
# https://keras.io/models/sequential/
#mod = Sequential()

# Add a layer
# https://keras.io/layers/about-keras-layers/
# https://keras.io/activations/
#mod.add(Conv2D(64, kernel_size=IMG_PATCH_SIZE, activation='relu',input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
#mod.add(Flatten())
#mod.add(Dense(2, activation='softmax'))
        
        
# https://keras.io/optimizers/
# https://keras.io/losses/
#mod.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy']) #f1 = 0.43
#mod.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy']) #f1 = 0.43

In [5]:
model = model_Lidar((16, 16, 3))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [6]:
TEST_SIZE = 0.33 # To be studied

# Step 0: Shuffle samples
np.random.seed(0)
np.random.shuffle(img)
np.random.seed(0)
np.random.shuffle(gt)

# Step 1: Split into validation and training set     
Xtrain, Xtest, gt_train, gt_test = train_test_split(img, gt, test_size=TEST_SIZE, random_state=1)

# Step 2: Give weights to classes ?
c_weight = {1: 3., 
            0: 1.}

# Step 3: Generate Generators
train_datagenerator = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
test_datagenerator  = ImageDataGenerator()

train_generator = train_datagenerator.flow(Xtrain, gt_train, batch_size=BATCH_SIZE)
test_generator  = test_datagenerator.flow(Xtest, gt_test, batch_size=BATCH_SIZE)

# Step 4: Early stop
early_stop_callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=0, mode='max', restore_best_weights=True)

# Step 5: Training
model.fit_generator(train_generator,
            validation_data=test_generator,
            steps_per_epoch=math.ceil(TRAINING_SIZE / BATCH_SIZE), #len(train_generator)/16,
            epochs=NUM_EPOCHS,
            callbacks = [early_stop_callback],
            class_weight=c_weight,
            validation_steps=math.ceil(len(test_generator)/BATCH_SIZE))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





4/4 [==============================] - 133s 33s/step - loss: 0.9083 - acc: 0.7188 - val_loss: 0.6783 - val_acc: 0.7729
Epoch 2/100
4/4 [==============================] - 122s 30s/step - loss: 1.0764 - acc: 0.7500 - val_loss: 0.6475 - val_acc: 0.7774
Epoch 3/100
4/4 [==============================] - 104s 26s/step - loss: 0.9155 - acc: 0.8281 - val_loss: 0.6688 - val_acc: 0.8049
Epoch 4/100
4/4 [==============================] - 116s 29s/step - loss: 0.8720 - acc: 0.8594 - val_loss: 0.6599 - val_acc: 0.7896
Epoch 5/100
4/4 [==============================] - 99s 25s/step - loss: 0.9682 - acc: 0.7969 - val_loss: 0.6282 - val_acc: 0.7988
Epoch 6/100
4/4 [==============================] - 103s 26s/step - loss: 1.1376 - acc: 0.7031 - val_loss: 0.6489 - val_acc: 0.8018
Epoch 7/100
4/4 [==============================] - 98s 25s/step - loss: 0.9928 - acc: 0.7812 - val_loss: 0.6722 -

In [12]:
from predictions import *

In [14]:
train_data_filename = "../Data/training/images/"

prediction_training_dir = "predictions_training_lidar_model_LEO/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)

for i in range(1, TRAINING_SIZE+1):
    pimg = get_prediction_with_groundtruth(train_data_filename, i, model)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(train_data_filename, i, model)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")


# serialize model to JSON
OUTPUT_FILENAME = "simple_model"
model_json = model.to_json()
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

validation_data_prediction = mod.predict_classes(data_te)
validation_label = []
for e in label_te:
    if (e[0] == 0):
        validation_label.append(1)
    else:
        validation_label.append(0)
        
validation_label = np.array(validation_label)     
print("F1 score = "+str(f1_score(validation_data_prediction, validation_label, labels=['1'])))

NameError: name 'label_to_img' is not defined